In [2]:
# GET PARAMETERS
import os 
import sys
# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.train_and_visu_non_recurrent import evaluate_config
from plotting.plotting import error_per_station_calendar_pattern

Training and Hyper-parameter tuning with Ray is not possible


## K-Fold Validation sur une config 'qui fait sens':

In [ ]:
from examples.train_model_on_k_fold_validation import train_model_on_k_fold_validation

trial_id = 'subway_in_config_good'
save_folder = 'K_fold_validation/training_with_HP_tuning/re_validation'
epochs = 300
train_model_on_k_fold_validation(trial_id,load_config =False,save_folder=save_folder,epochs=epochs,hp_tuning_on_first_fold= True)

## K-Fold Validation sur une config d'HP tuning
- 500 samples
- 100 epochs max pour HP 
- grace period = 20 epochs 
- 100 epochs validations

In [1]:
from examples.train_model_on_k_fold_validation import train_model_on_k_fold_validation

trial_id  = 'subway_in_netmob_POIs_STGCN_VariableSelectionNetwork_MSELoss_2025_01_07_23_12_41192'
# 'subway_in_netmob_POIs_STGCN_VariableSelectionNetwork_MSELoss_2025_01_07_23_12_41192
# 'subway_in_netmob_POIs_STGCN_VariableSelectionNetwork_MSELoss_2025_01_07_05_04_80480'
# 'subway_in_STGCN_MSELoss_2025_01_06_08_00_94523'
# 'subway_in_subway_out_STGCN_VariableSelectionNetwork_MSELoss_2025_01_06_02_04_17963'

save_folder = 'K_fold_validation/training_with_HP_tuning/re_validation'
epochs = 100
train_model_on_k_fold_validation(trial_id,load_config =True,save_folder=save_folder,epochs=epochs,hp_tuning_on_first_fold= True)

Training and Hyper-parameter tuning with Ray is not possible

>>>> Load best CONFIG

Init Dataset:  torch.Size([7392, 40])
Number of Nan Value:  tensor(0)
Total Number of Elements:  295680 

U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  AMP
U/Utarget size: torch.Size([3420, 34, 7])/torch.Size([3420, 34, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  BEL
U/Utarget size: torch.Size([3420, 36, 7])/torch.Size([3420, 36, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  BRO
U/Utarget size: torch.Size([3420, 30, 7])/torch.Size([3420, 30, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  COR
U/Utarget size: torch.Size([3420, 39, 7])/torch.Size([3420, 39, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  CUI
U/Utarget size: torch.Size([3420, 67, 7])/torch.Size([3420, 67, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  CUS
U/Utarget size: torch.Size([3420, 21, 7])/torch.Size([3420, 21, 1]) Train/Valid/Test 2052 684 683
Spatial u

/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


epoch: 0 
 min\epoch : 0.08
Estimated time for training: 4.9min 

Training Throughput:508.84 sequences per seconds
>>> Training complete in: 0:03:46.871814
>>> Training performance time: min 0.05408287048339844 avg 0.060576677322387695 seconds (+/- 0.002922273275985725)
>>> Loading performance time: min 0.002754688262939453 avg 0.04361154214982133 seconds (+/- 0.09782189505763646)
>>> Forward performance time: 0.022784450779790463 seconds (+/- 0.0007972278031056888)
>>> Backward performance time: 0.032222762572026886 seconds (+/- 0.002536123713693241)
>>> Plotting performance time: 3.7063251842151987e-06 seconds (+/- 7.682438342801457e-06)
>>> Saving performance time: 0.8139693439006805 seconds (+/- 0.14637781744793485)
>>> PI-tracking performance time: 5.9989967731514365e-06 seconds (+/- 3.7989669971263714e-06)
>>> Scheduler-update performance time: 4.420617614129577e-05 seconds (+/- 1.9924805996021285e-05)
>>> Peak Power during training: 109.889 W)
>>> Validation time: 0:00:00.223879

/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


epoch: 0 
 min\epoch : 0.05
Estimated time for training: 4.8min 

Training Throughput:512.44 sequences per seconds
>>> Training complete in: 0:03:33.814631
>>> Training performance time: min 0.05532503128051758 avg 0.06070232391357422 seconds (+/- 0.0034485886436542513)
>>> Loading performance time: min 0.0027947425842285156 avg 0.04005344402749568 seconds (+/- 0.07005765355658178)
>>> Forward performance time: 0.022764741571741225 seconds (+/- 0.0006968391968838845)
>>> Backward performance time: 0.03248190723872726 seconds (+/- 0.0031386297597366148)
>>> Plotting performance time: 4.7659633135554766e-06 seconds (+/- 1.8428872590775173e-05)
>>> Saving performance time: 0.7870516379674276 seconds (+/- 0.17964614054447794)
>>> PI-tracking performance time: 5.416195802014283e-06 seconds (+/- 2.9892093439933175e-06)
>>> Scheduler-update performance time: 4.1598021382033226e-05 seconds (+/- 1.6557992417324335e-05)
>>> Peak Power during training: 110.357 W)
>>> Validation time: 0:00:00.2261

...

epoch: 0 
 min\epoch : 0.05
Estimated time for training: 4.7min 

Training Throughput:515.11 sequences per seconds
>>> Training complete in: 0:03:38.133760
>>> Training performance time: min 0.05587482452392578 avg 0.06060028076171875 seconds (+/- 0.002364476301339621)
>>> Loading performance time: min 0.0028083324432373047 avg 0.04177778574510447 seconds (+/- 0.08537520638176577)
>>> Forward performance time: 0.022854879297812016 seconds (+/- 0.000699586085001377)
>>> Backward performance time: 0.03210469235269415 seconds (+/- 0.002024241220716932)
>>> Plotting performance time: 3.923069347034801e-06 seconds (+/- 7.505665337442242e-06)
>>> Saving performance time: 0.7990272045135498 seconds (+/- 0.15788685781434617)
>>> PI-tracking performance time: 5.683513602825126e-06 seconds (+/- 3.887855401565398e-06)
>>> Scheduler-update performance time: 4.17449257590554e-05 seconds (+/- 1.749978822924323e-05)
>>> Peak Power during training: 112.519 W)
>>> Validation time: 0:00:00.225463
Propor

/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


epoch: 0 
 min\epoch : 0.05
Estimated time for training: 4.7min 

Training Throughput:513.31 sequences per seconds
>>> Training complete in: 0:03:40.125915
>>> Training performance time: min 0.05434417724609375 avg 0.060645103454589844 seconds (+/- 0.0032550617662726053)
>>> Loading performance time: min 0.0027780532836914062 avg 0.04214973051892717 seconds (+/- 0.08761121141684215)
>>> Forward performance time: 0.022732931818123643 seconds (+/- 0.0005798858648761706)
>>> Backward performance time: 0.03245951616566839 seconds (+/- 0.0029124505351533383)
>>> Plotting performance time: 3.809880728673453e-06 seconds (+/- 1.131639232295554e-05)
>>> Saving performance time: 0.804086841069735 seconds (+/- 0.15441530195074157)
>>> PI-tracking performance time: 5.70037148215554e-06 seconds (+/- 3.183443085622428e-06)
>>> Scheduler-update performance time: 4.231568538781369e-05 seconds (+/- 1.7547536863274343e-05)
>>> Peak Power during training: 112.762 W)
>>> Validation time: 0:00:00.226533
Pr

/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


epoch: 0 
 min\epoch : 0.10
Estimated time for training: 10.3min 

Training Throughput:514.35 sequences per seconds
>>> Training complete in: 0:08:58.067880
>>> Training performance time: min 0.05358004570007324 avg 0.06086897850036621 seconds (+/- 0.003309925264271607)
>>> Loading performance time: min 0.0015916824340820312 avg 0.04067627246697671 seconds (+/- 0.0970984349954954)
>>> Forward performance time: 0.022913523024092237 seconds (+/- 0.0011325457022341278)
>>> Backward performance time: 0.032552704038501135 seconds (+/- 0.0028756586373772067)
>>> Plotting performance time: 4.260226933643071e-06 seconds (+/- 7.27573499565625e-06)
>>> Saving performance time: 1.1523899670803186 seconds (+/- 0.20751482643025912)
>>> PI-tracking performance time: 9.377797444661459e-06 seconds (+/- 5.379875573957895e-06)
>>> Scheduler-update performance time: 6.116038621074022e-05 seconds (+/- 2.8039933910978547e-05)
>>> Peak Power during training: 113.291 W)
>>> Validation time: 0:00:00.552882
Pr

## Gestion des outliers: Estimation du Flux Max d'une station
- De Nombreux outliers présents à Perrache, d'autres assez élevée sur les autres stations. On va essayer de gérer ces outliers. 
    - On prend un Flux maximal enregistré : le Flux in max en 15min enregistré pendant la fête des lumière
        - Le Flux max est enregistré à Bellecour, autour de 2500 personnes / 15 minutes
        - Le second Flux max est enregistré à Charpennes, autour de 1600 personnes / 15 minutes
        - Le 3eme Flux max est enregsitré à Gorge de Loup, autour de 1600 personnes / 15 minutes
        - Aucunes des autres stations n'a dépassée 1400, même pendant la fête des lumières. 

- On choisit donc d'inputer la données, par interpolation linéaire, pour tout les flux dépassant:
    -  2500 pour bellecour
    - 1600 pour charpnnes et gorge de Loup
    - 1400 pour les autres stations.

In [ ]:
trial_id = 'subway_in_netmob_POIs_STGCN_VariableSelectionNetwork_MSELoss_2025_01_07_05_04_80480'
# "subway_in_subway_out_STGCN_VariableSelectionNetwork_MSELoss_2025_01_06_02_04_17963"
# 'subway_in_netmob_POIs_STGCN_VariableSelectionNetwork_MSELoss_2025_01_07_05_04_80480'
# "subway_in_STGCN_MSELoss_2025_01_06_08_00_94523"
args,_ = load_configuration(trial_id,load_config=True,epochs=None)

# Comparaison des flux avec ce qu'on peut trouver sur les validations individuelles:

In [ ]:
import pandas as pd
import glob
import os 
import sys
# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from constants.paths import FOLDER_PATH

list_files = glob.glob(f"{parent_dir}/{FOLDER_PATH}/validations_individuelles/*.txt")

# Load df :


path_txt = list_files[1]
df = pd.read_csv(path_txt, sep = "\t")
df['is_subway'] = df.EMP_CEB.transform(lambda x : x[:3])
df_subway = df[df['is_subway'] == 'VMM']   # Extract Funicular and Subway, starting both by "VMM"

df_subway['subway_mode'] = df_subway.EMP_CEB.transform(lambda x : x[:4])
df_subway = df_subway[df_subway['subway_mode'] != 'VMMF'] # Extract VVMx and remove VMMF (i.e. Subway Lane, excluding Funicular)
df_subway.VAL_DATE = pd.to_datetime(df_subway.VAL_DATE)

if False:
    # Load ref subway
    ref_subway = pd.read_csv(f"{folder_path}/ref_subway.csv").rename(columns = {'MEAN_X' : 'lon','MEAN_Y':'lat'})
    ref_subway = ref_subway[['lon','lat','COD_TRG','LIB_STA_SIFO']]

#Get Subway  
df_subway['subway_mode'] = df_subway.EMP_CEB.transform(lambda x : x[:4])
df_subway = df_subway[df_subway['subway_mode'] != 'VMMF'] # Extract VVMx (i.e. Subway Lane)
df_subway['COD_TRG'] = df_subway.EMP_CEB.transform(lambda x : x[5:8])
df_subway['lane'] = df_subway.EMP_CEB.transform(lambda x : x[3])

In [ ]:
df_subway_agg = df_subway.groupby([pd.Grouper(freq='15min',key='VAL_DATE'),'COD_TRG']).count().iloc[:,0]
df_subway_agg.name = 'Flow'
df_subway_agg = df_subway_agg.reset_index()

#df_subway_agg[~df_subway_agg['COD_TRG'].isin(['CHA','BEL','GOR'])].pivot_table(index = 'VAL_DATE',columns = 'COD_TRG',values='Flow').plot()
df_subway_agg[df_subway_agg['COD_TRG'].isin(['HOT','BEL','CHA','AMP','PER','PAR'])].pivot_table(index = 'VAL_DATE',columns = 'COD_TRG',values='Flow').plot()
#df_subway_agg.pivot_table(index = 'VAL_DATE',columns = 'COD_TRG',values='Flow').plot()

## Génération de la configuration, choix des modification, entrainement et visalisation.
Args: 
--
- station : définie les station qui vont être regardée (visualisation profil time série réelle et prédiction)
- set_spatial_units: défini les stations qui vont être prise en compte lors de l'entrainement. La taille de 'set_spatial_unit' est donc la taille du graphe. Si non précisé, on fait la prédiction sur les 40 STATION du réseau.

In [4]:
# Init:
dataset_names = ["subway_in"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'
station = ['BEL','PAR','AMP','SAN','FLA']   # 'BON'  #'GER'
# ...

# Modif 
modification = {'epochs' : 300,
  'lr':1.15e-3,
  'weight_decay':0.029826,
  'dropout': 0.303705,
  'temporal_h_dim': 16,
  'spatial_h_dim': 8,
  'output_h_dim': 16,	
  'scheduler': None,
 }
# ...

# Training and visu: 
training_mode_to_visualise = ['test'] # ['test','valid','train']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(model_name,dataset_names,dataset_for_coverage,
                                                   station=station,modification=modification,
                                                   training_mode_to_visualise=training_mode_to_visualise)

>>>>Model: STGCN; K_fold = 6; Loss function: MSE 



Init Dataset:  torch.Size([7392, 40])
Number of Nan Value:  tensor(0)
Total Number of Elements:  295680 

U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
----------------------------------------
Fold n°0

Init Dataset:  torch.Size([1183, 40])
Number of Nan Value:  tensor(0)
Total Number of Elements:  47320 

U/Utarget size: torch.Size([1087, 40, 7])/torch.Size([1087, 40, 1]) Train/Valid 815 271
Model size: 0.000GB
number of total parameters: 12321
number of trainable parameters: 12321

start training
epoch: 0 
 min\epoch : 0.02
Estimated time for training: 3.3min 

Training Throughput:1780.26 sequences per seconds
>>> Training complete in: 0:02:43.842861
>>> Training performance time: min 0.012731790542602539 avg 0.01723313331604004 seconds (+/- 0.002395032055668244)
>>> Loading performance time: min 0.00011539459228515625 avg 0.010403499856416887 seconds (+/- 0.017553099996201867)
>>> Forward performance time: 0.004535740438921816 sec

Loading BokehJS ...

## Best Config Subway-IN  sur 500 Sample 

In [5]:
# Init:
dataset_names = ["subway_in","subway_out"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'
station = ['BEL','PAR','AMP','SAN','FLA']   # 'BON'  #'GER'
# ...

# Modif 
modification = {'epochs' : 100,
  'lr':1.45e-3,
  'weight_decay':0.060384,
  'dropout': 0.368853,
  'temporal_h_dim': 16,
  'spatial_h_dim': 256,
  'output_h_dim': 256,	
  'scheduler': None,
    'vision_model_name': 'VariableSelectionNetwork',
    'vision_concatenation_early':True,   
    'vision_concatenation_late':True,
    'vision_num_heads':3,
    'vision_grn_out_dim':12,
 }
# ...

# Training and visu: 
training_mode_to_visualise = ['test'] # ['test','valid','train']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(model_name,dataset_names,dataset_for_coverage,
                                                   station=station,modification=modification,
                                                   training_mode_to_visualise=training_mode_to_visualise)

>>>>Model: STGCN; K_fold = 6; Loss function: MSE 

Init Dataset:  torch.Size([7392, 40])
Number of Nan Value:  tensor(0)
Total Number of Elements:  295680 

U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  AMP
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  BEL
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  BRO
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  COR
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  CUI
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  CUS
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  FLA
U/Utarget size: torch.Si

Loading BokehJS ...

## Best Config Subway-IN / Subway-OUT sur 500 Sample 
Uitlisation de cette config sur 'netmob_POIs'. Même ordre de grandeur de paramètres, on pourrait avoir le même type d'HP tuning (réponse: Non)

In [2]:
# Init:
dataset_names = ["subway_in","netmob_POIs"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'
station = ['BEL','PAR','AMP','SAN','FLA']   # 'BON'  #'GER'
# ...

# Modif 
modification = {'epochs' : 100,
  'lr':1.45e-3,
  'weight_decay':0.060384,
  'dropout': 0.368853,
  'temporal_h_dim': 16,
  'spatial_h_dim': 256,
  'output_h_dim': 256,	
  'scheduler': None,
    'vision_model_name': 'VariableSelectionNetwork',
    'vision_concatenation_early':True,   
    'vision_concatenation_late':True,
    'vision_num_heads':3,
    'vision_grn_out_dim':12,
 }
# ...

# Training and visu: 
training_mode_to_visualise = ['test'] # ['test','valid','train']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(model_name,dataset_names,dataset_for_coverage,
                                                   station=station,modification=modification,
                                                   training_mode_to_visualise=training_mode_to_visualise)

NameError: name 'evaluate_config' is not defined

## Best Config Subway-IN / Subway-OUT sur 500 Sample 

In [ ]:
# GET PARAMETERS
import os 
import sys
# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.train_and_visu_non_recurrent import evaluate_config
from plotting.plotting import error_per_station_calendar_pattern

# Init:
dataset_names = ["subway_in","subway_out"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'
station = ['BEL','PAR','AMP','SAN','FLA']   # 'BON'  #'GER'
# ...

# Modif 
modification = {'epochs' : 100, #100
                'lr':4e-4,# 4e-4,
                'weight_decay':0.05,
                'dropout':0.1,
                #'set_spatial_units':  station,  
                #   
                #'scheduler':None,
                'scheduler':True,
                'torch_scheduler_milestone': 2,
                'torch_scheduler_gamma':0.99,
                'torch_scheduler_lr_start_factor':0.1,

                'vision_num_heads':4,
                "vision_grn_out_dim":32,
                'vision_model_name': 'VariableSelectionNetwork',
                'vision_concatenation_early':True,   
                'vision_concatenation_late':True,
                           }
# ...

# Training and visu: 
training_mode_to_visualise = ['test'] # ['test','valid','train']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(model_name,dataset_names,dataset_for_coverage,
                                                   station=station,modification=modification,
                                                   training_mode_to_visualise=training_mode_to_visualise)

## Pattern d'erreur selon les stations : 

In [ ]:
# Init
for training_mode in training_mode_to_visualise:
    min_flow = 20  # Minimal Flow considered for MAPE, otherwise set error = 0
    limit_percentage_error = 200 # 300% plus mauvais que quand on se sert du previous 
    fig,axes = error_per_station_calendar_pattern(trainer,ds,training_mode,metrics = ['mse','mae','mape','previous_value'],
                                                freq='1h',
                                                min_flow=min_flow,
                                                figsize = (30,5*len(ds.spatial_unit)),
                                                limit_percentage_error = limit_percentage_error)

In [1]:
# GET PARAMETERS
import os 
import sys
# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.train_and_visu_non_recurrent import evaluate_config
from plotting.plotting import error_per_station_calendar_pattern

dataset_names = ["subway_in","subway_out"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'

station = ['BEL','PAR','AMP','FLA']   # 'BON'  #'GER'
# ...

modification = {'epochs' : 200, #100
                'lr':4e-4,
                #'set_spatial_units': station,#['CHA','GER','BON','SOI'],
                #'TE_concatenation_late':False,
                #'TE_concatenation_early':True,   
                #'TE_embedding_dim' : 16, # 3
                #'TE_multi_embedding' : True , 
                'vision_model_name': 'VariableSelectionNetwork',
                'vision_concatenation_early':True,   
                'vision_concatenation_late':True,
                           }
training_mode_to_visualise = ['test']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(model_name,dataset_names,dataset_for_coverage,
                                                   station = station,modification=modification,training_mode_to_visualise=training_mode_to_visualise)

# Init
if False:
    for training_mode in training_mode_to_visualise:
        min_flow = 20  # Minimal Flow considered for MAPE, otherwise set error = 0
        limit_percentage_error = 200 # 300% plus mauvais que quand on se sert du previous 
        fig,axes = error_per_station_calendar_pattern(trainer,ds,training_mode,metrics = ['mse','mae','mape','previous_value'],
                                                    freq='1h',
                                                    min_flow=min_flow,
                                                    figsize = (30,5*len(ds.spatial_unit)),
                                                    limit_percentage_error = limit_percentage_error)

Training and Hyper-parameter tuning with Ray is not possible
>>>>Model: STGCN; K_fold = 6; Loss function: MSE 

Init Dataset:  torch.Size([7392, 40])
Number of Nan Value:  tensor(0)
Total Number of Elements:  295680 

U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  AMP
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  BEL
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  BRO
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  COR
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  CUI
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/Test 2052 684 683
Spatial unit:  CUS
U/Utarget size: torch.Size([3420, 40, 7])/torch.Size([3420, 40, 1]) Train/Valid/

Loading BokehJS ...

In [ ]:
trainer.args.epochs = 20
trainer.train_and_valid(mod = 1000,mod_plot = None) 
print(trainer.performance['valid_metrics']['mse'],trainer.performance['valid_metrics']['mae'],trainer.performance['valid_metrics']['mape'])
print(trainer.performance['test_metrics']['mse'],trainer.performance['test_metrics']['mae'],trainer.performance['test_metrics']['mape'])

import pandas as pd 
pd.DataFrame(dict(valid = trainer.valid_loss)).iloc[-120:].plot()

In [ ]:
# GET PARAMETERS
import os 
import sys
# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.train_and_visu_non_recurrent import evaluate_config
from plotting.plotting import error_per_station_calendar_pattern

dataset_names = ["subway_in","subway_out"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'

station = ['BEL','PAR','AMP','FLA']   # 'BON'  #'GER'
# ...

modification = {'epochs' : 100, #100
                'lr':4e-4,
                #'set_spatial_units': station,#['CHA','GER','BON','SOI'],
                #'TE_concatenation_late':False,
                #'TE_concatenation_early':True,   
                #'TE_embedding_dim' : 16, # 3
                #'TE_multi_embedding' : True , 
                'vision_model_name': 'VariableSelectionNetwork',
                'vision_concatenation_early':False,   
                'vision_concatenation_late':True,
                           }
training_mode_to_visualise = ['test']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(model_name,dataset_names,dataset_for_coverage,
                                                   station = station,modification=modification,training_mode_to_visualise=training_mode_to_visualise)

# Init
if False:
    for training_mode in training_mode_to_visualise:
        min_flow = 20  # Minimal Flow considered for MAPE, otherwise set error = 0
        limit_percentage_error = 200 # 300% plus mauvais que quand on se sert du previous 
        fig,axes = error_per_station_calendar_pattern(trainer,ds,training_mode,metrics = ['mse','mae','mape','previous_value'],
                                                    freq='1h',
                                                    min_flow=min_flow,
                                                    figsize = (30,5*len(ds.spatial_unit)),
                                                    limit_percentage_error = limit_percentage_error)

In [ ]:
print(trainer.performance['valid_metrics']['mse'],trainer.performance['valid_metrics']['mae'],trainer.performance['valid_metrics']['mape'])
print(trainer.performance['test_metrics']['mse'],trainer.performance['test_metrics']['mae'],trainer.performance['test_metrics']['mape'])

In [ ]:
trainer.train_and_valid(mod = 1000,mod_plot = None) 
print(trainer.performance['valid_metrics']['mse'],trainer.performance['valid_metrics']['mae'],trainer.performance['valid_metrics']['mape'])
print(trainer.performance['test_metrics']['mse'],trainer.performance['test_metrics']['mae'],trainer.performance['test_metrics']['mape'])

In [ ]:
from plotting.plotting import plot_attn_weight
from constants.paths import CALENDAR_TYPE
nb_calendar_data = len(CALENDAR_TYPE)

training_mode = 'test'
temporal_agg = 'weekday_hour' # 'hour' # 'weekday' # 'weekday_hour'
save = 'attn_weight'
plot_attn_weight(trainer,nb_calendar_data,ds,training_mode,temporal_agg,save)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Supposons que attn_weights est votre tenseur de poids d'attention
# attn_weights : Shape [B, 1, P]
# Prenons le premier exemple du batch
attn_weights_example = attn_weights[0].squeeze(0)  # Shape [P]

# Normaliser les poids d'attention pour la visualisation
attn_weights_example = attn_weights_example.detach().cpu().numpy()  # Convertir en numpy

# Créer une figure
plt.figure(figsize=(10, 6))

# Afficher les poids d'attention
plt.bar(range(len(attn_weights_example)), attn_weights_example, color='skyblue')
plt.title('Poids d\'attention pour l\'exemple 1')
plt.xlabel('Séries temporelles')
plt.ylabel('Poids d\'attention')

# Ajouter des ticks pour les séries temporelles
plt.xticks(range(len(attn_weights_example)), [f'Série {i}' for i in range(len(attn_weights_example))], rotation=45)

# Afficher la figure
plt.tight_layout()
plt.show()

In [ ]:
from examples.train_and_visu_non_recurrent import evaluate_config
from plotting.plotting import error_per_station_calendar_pattern

dataset_names = ["subway_in"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'

station = ['PAR']   # 'BON'  #'GER'

modification = {'epochs' : 250, #100
                'lr':1e-4,
                #'set_spatial_unit': station
                }

training_mode_to_visualise = ['test','valid','train']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(model_name,dataset_names,dataset_for_coverage,
                                                  station = station,
                                                  modification=modification,
                                                  training_mode_to_visualise=training_mode_to_visualise)

In [ ]:
# Init
for training_mode in training_mode_to_visualise:
    min_flow = 20  # Minimal Flow considered for MAPE, otherwise set error = 0
    limit_percentage_error = 200 # 300% plus mauvais que quand on se sert du previous 
    fig,axes = error_per_station_calendar_pattern(trainer,ds,training_mode,metrics = ['mse','mae','mape','previous_value'],
                                                freq='1h',
                                                min_flow=min_flow,
                                                figsize = (30,5*len(ds.spatial_unit)),
                                                limit_percentage_error = limit_percentage_error)

In [ ]:
import torch
import pandas as pd
index_perrache = list(ds.spatial_unit).index('PER')
inputs = [[x,y,x_c] for  x,y,x_c in ds.dataloader['test']]
X = torch.cat([x for x,_,_ in inputs],0)
Y = torch.cat([y for _,y,_ in inputs],0)
#X_c = torch.cat([x_c for x,_,_ in inputs],-1)
l = 96
pd.DataFrame({'PER_TRUE': Y[:,index_perrache,0].detach().clone().reshape(-1),
              #'t-1': X[:,index_perrache,-1].detach().clone().reshape(-1),
              #'t-2': X[:,index_perrache,-2].detach().clone().reshape(-1),
              #'t-3': X[:,index_perrache,-3].detach().clone().reshape(-1),
              #'t-4': X[:,index_perrache,-4].detach().clone().reshape(-1),
              #'t-5': X[:,index_perrache,-5].detach().clone().reshape(-1),
              #'t-6': X[:,index_perrache,-6].detach().clone().reshape(-1),
              't-d': X[:,index_perrache,-7].detach().clone().reshape(-1),
              }).iloc[l:l+7*l].plot()


In [ ]:
from examples.train_and_visu_non_recurrent import evaluate_config
dataset_names = ["subway_in","calendar"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'

station = 'GER'  # 'BON'  #'GER'
apps = ['Google_Maps']# ['Instagram','Twitter','Google_Maps'] # 'Instagram'  # 'Twitter' # 'Google_Maps' # 'Facebook'


transfer_modes = ['DL'] # ['DL'] # ['UL'] # ['DL','UL']
type_POIs = ['stadium'] #['stadium','station'] #['stadium','nightclub'] #['stadium']
spatial_units = ['Matmut Stadium Gerland','GER']  #spatial_units = ['Lou_rugby']  # ['Astroballe'] #['Lou_rugby','Ninkasi_Kao'] #['Lou_rugby'] #['Ninkasi_Kao'] 
expanded = '_expanded' # '' # '_expanded' # ''
POI_or_stations = ['POI']# ['POI','station'] # 'station'
modification = {'epochs' : 400, #100
                'lr':1e-4,
                'set_spatial_units': ['CHA','GER','BON','SOI'],#['CHA','GER','BON','SOI'],
                'TE_concatenation_late':False,
                'TE_concatenation_early':True,   
                'TE_embedding_dim' : 2,
                'TE_multi_embedding' : True ,         
                           }
training_mode_to_visualise = ['test']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(model_name,dataset_names,dataset_for_coverage,transfer_modes= transfer_modes,
                                                   type_POIs = type_POIs,spatial_units = spatial_units,apps = apps,POI_or_stations = POI_or_stations,
                                                   expanded=expanded,modification=modification,training_mode_to_visualise=training_mode_to_visualise)

## Evaluate on non recurrent event: 
### Visualisation: 
#### Cas `subway_in` stade du Lou Gerland avec un match de Rugby : 

## HP tuning: 

In [ ]:
from examples.Total_evaluation_of_model import HP_and_valid_one_config,hyperparameter_tuning
from examples.benchmark import local_get_args
import pickle

model_name = 'STGCN' #'CNN'
dataset_for_coverage = ['subway_in','netmob_POIs'] 
dataset_names = ['calendar']
vision_model_name = None

args,_,_ = local_get_args(model_name,
                        args_init = None,
                        dataset_names=dataset_names,
                        dataset_for_coverage=dataset_for_coverage,
                        modification = {'ray':True,
                                        'grace_period':2,
                                        'HP_max_epochs':10,
                                        'evaluate_complete_ds' : True,
                                        'set_spatial_units' : ['BON','SOI','GER','CHA'],
                                        'vision_model_name': None
                                        })

# Init 
epochs_validation = 30
num_samples = 10

# HP and evaluate K-fold best config
#HP_and_valid_one_config(args,epochs_validation,num_samples)
analysis,trial_id = hyperparameter_tuning(args,num_samples)
print('trial_id: ',trial_id)



## Load HP tuning, have a look on the best configurations:

In [1]:
import pickle 
import pandas as pd 

# 2000 Trial et 100 Epochs: 
trial_ids = ['subway_in_subway_out_STGCN_VariableSelectionNetwork_MSELoss_2025_01_06_02_04_17963','subway_in_STGCN_MSELoss_2025_01_06_08_00_94523']

# 'subway_in_STGCN_MSELoss_2024_12_11_00_16_42861'   -> 'Seulement' 400 trials, et 50 epochs 
for trial_id in trial_ids:
    columns_to_drop = ['time_this_iter_s', 'done', 'date', 
                    'timestamp', 'time_total_s', 'pid', 'hostname', 'node_ip',
                    'time_since_restore', 'iterations_since_restore', 'experiment_tag']
    path_csv = f'save/HyperparameterTuning/{trial_id}.csv'
    path_pickle = 'save/HyperparameterTuning/model_args.pkl'

    df_hp_tuning = pd.read_csv(path_csv)
    model_args = pickle.load(open(path_pickle,'rb'))


    metric = 'Loss_model'

    dict_rename = {key: key.split('/')[-1] for key in df_hp_tuning.columns}
    df_hp_tuning = df_hp_tuning.rename(columns = dict_rename)
    df_hp_tuning = df_hp_tuning.drop(columns = columns_to_drop)


    k=10
    print(df_hp_tuning.shape)
    display(df_hp_tuning.sort_values(metric).iloc[:k])



(500, 17)


,trial_id,training_iteration,Loss_model,lr,weight_decay,dropout,temporal_h_dim,spatial_h_dim,output_h_dim,scheduler,concatenation_early,concatenation_late,num_heads,grn_out_dim,torch_scheduler_milestone,torch_scheduler_gamma,torch_scheduler_lr_start_factor
160,87851_00160,100,0.005920,0.00145,0.060384,0.368853,16,256,256,NaN,True,True,3,12,NaN,NaN,NaN
451,87851_00451,100,0.006093,0.00125,0.009742,0.491189,32,128,64,True,False,True,1,4,5.0,0.992034,0.828268
69,87851_00069,100,0.006127,0.00455,0.014063,0.106061,8,32,8,True,False,True,2,8,27.0,0.990475,0.145083
259,87851_00259,100,0.006154,0.00060,0.073119,0.390112,16,64,32,True,False,True,6,24,18.0,0.987133,0.781067
20,87851_00020,100,0.006339,0.00225,0.034687,0.529574,32,32,16,True,False,True,1,16,17.0,0.996583,0.589259
53,87851_00053,100,0.006392,0.00035,0.068259,0.275934,16,256,16,NaN,True,False,1,16,NaN,NaN,NaN
176,87851_00176,100,0.006405,0.00025,0.034697,0.060886,256,64,32,True,False,True,6,24,25.0,0.997865,0.714208
359,87851_00359,100,0.006409,0.00175,0.014504,0.131933,256,256,64,True,True,False,6,192,17.0,0.987810,0.487705
121,87851_00121,100,0.006574,0.00120,0.025116,0.568096,32,128,64,NaN,True,False,2,8,NaN,NaN,NaN
189,87851_00189,100,0.006692,0.00365,0.015256,0.285538,32,8,256,True,True,False,2,64,3.0,0.997815,0.115074


(500, 13)


,trial_id,training_iteration,Loss_model,lr,weight_decay,dropout,temporal_h_dim,spatial_h_dim,output_h_dim,scheduler,torch_scheduler_milestone,torch_scheduler_gamma,torch_scheduler_lr_start_factor
74,d02e7_00074,100,0.005833,0.00110,0.029826,0.303705,16,8,16,NaN,NaN,NaN,NaN
425,d02e7_00425,100,0.005834,0.00070,0.032901,0.116875,8,128,16,True,19.0,0.995579,0.657189
430,d02e7_00430,100,0.005911,0.00445,0.053486,0.550622,64,8,128,NaN,NaN,NaN,NaN
497,d02e7_00497,100,0.005931,0.00250,0.094097,0.430751,32,64,16,NaN,NaN,NaN,NaN
3,d02e7_00003,100,0.005935,0.00055,0.090834,0.148635,32,32,256,NaN,NaN,NaN,NaN
13,d02e7_00013,100,0.006026,0.00205,0.068388,0.112663,8,128,8,NaN,NaN,NaN,NaN
434,d02e7_00434,100,0.006069,0.00090,0.057198,0.481282,32,256,8,NaN,NaN,NaN,NaN
38,d02e7_00038,100,0.006073,0.00240,0.050927,0.525008,16,8,256,NaN,NaN,NaN,NaN
451,d02e7_00451,100,0.006330,0.00055,0.029741,0.225671,16,128,256,True,23.0,0.997041,0.786742
384,d02e7_00384,100,0.006378,0.00100,0.047268,0.451278,128,16,256,NaN,NaN,NaN,NaN


In [ ]:
save_folder = 'save/K_fold_validation/training_with_HP_tuning'

for trial_id in trial_ids:
    print(f'\n{trial_id}')
    metric_csv = f'{save_folder}/METRICS_{trial_id}.csv'
    df_metrics_k_fold_valid = pd.read_csv(metric_csv,index_col =0)

    losses_csv = f'{save_folder}/Losses_{trial_id}.csv'
    df_losses_k_fold_valid = pd.read_csv(losses_csv,index_col =0)

    display(df_metrics_k_fold_valid)
    usefull_columns = [c for c in df_losses_k_fold_valid.columns if 'valid' in c]
    df_losses_k_fold_valid[usefull_columns].iloc[10:].plot()

In [ ]:
import numpy as np 
from examples.train_model_on_k_fold_validation import train_valid_K_models,get_model_metrics
from examples.benchmark import local_get_args


# TE modification: 
TE_modification = {'TE_concatenation_early':True,
                  'TE_multi_embedding':True,
                  'TE_concatenation_late': False,
                  'TE_out_h_dim':4,
                  'TE_embedding_dim':4
               }


for dataset_names,te_modif in zip([['subway_in','calendar'],['subway_in']],[TE_modification,{}]):
   ### === Modification :
   modification = {'evaluate_complete_ds':True,
                  'temporal_h_dim':32,
                  'spatial_h_dim':8,
                  'output_h_dim':64,
                  'dropout':5e-2,
                  'weight_decay':5e-3,
                  'lr':5e-4,
                  'epochs':2
                  }
   modification.update(te_modif)
   ### _____________________

   # Load Config
   args,_,_ = local_get_args(model_name= 'STGCN',
                           args_init=None,
                           dataset_names= dataset_names,
                           dataset_for_coverage= ['subway_in','netmob_POIs'] ,
                           modification=modification
                           )

   # Train, Valid: K-fold and complete ds 
   trainer,args,valid_losses,training_mode_list,metric_list,df_loss = train_valid_K_models(args,
                                                                                          folds=np.arange(args.K_fold),
                                                                                          hp_tuning_on_first_fold=True,
                                                                                          trial_id= None,
                                                                                          save_folder=None)
   # Get metrics: 
   df_results,df_metrics =  get_model_metrics(trainer,args,valid_losses,training_mode_list,metric_list)

   display(df_results)
   display(df_metrics)

   df_losses = pd.DataFrame({'Valid_loss':trainer.valid_loss,
               'Train_loss':trainer.train_loss
               })[10:]

   df_losses.plot()

In [ ]:
from examples.train_model_on_k_fold_validation import load_configuration
from examples.train_and_visu_non_recurrent import evaluate_config

trial_id = 'calendar_STGCN_MSELoss_2024_12_17_21_18_13719'
best_args,folds = load_configuration(trial_id,load_config=True,epochs=None)

dict_keys_embedding = {'TE_concatenation_order/concatenation_late':'TE_concatenation_late',
            'TE_concatenation_order/concatenation_early':'TE_concatenation_early',
             'TE_fc1/fc1':'TE_fc1', 
             'TE_fc1/fc2':'TE_fc2', 
             'TE_fc1/activation_fc1':'TE_activation_fc1',
             }

for key in dict_keys_embedding.keys():
    value = vars(best_args)[key]
    new_key = dict_keys_embedding[key]
    setattr(best_args,new_key,value)


modification = vars(best_args)


dataset_names = ["calendar"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'

station = 'GER'  # 'BON'  #'GER'
apps = ['Google_Maps']# ['Instagram','Twitter','Google_Maps'] # 'Instagram'  # 'Twitter' # 'Google_Maps' # 'Facebook'


transfer_modes = ['DL'] # ['DL'] # ['UL'] # ['DL','UL']
type_POIs = ['stadium'] #['stadium','station'] #['stadium','nightclub'] #['stadium']
spatial_units = ['Matmut Stadium Gerland','GER']  #spatial_units = ['Lou_rugby']  # ['Astroballe'] #['Lou_rugby','Ninkasi_Kao'] #['Lou_rugby'] #['Ninkasi_Kao'] 
expanded = '_expanded' # '' # '_expanded' # ''
POI_or_stations = ['POI']# ['POI','station'] # 'station'

(trainer,ds,ds_no_shuffle,args) = evaluate_config(model_name,dataset_names,dataset_for_coverage,transfer_modes= transfer_modes,
                                                   type_POIs = type_POIs,spatial_units = spatial_units,apps = apps,POI_or_stations = POI_or_stations,
                                                   expanded=expanded,modification=modification,station=station)

In [ ]:
dataset_names = ["calendar"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'
vision_model_name =  None

station = 'GER'  # 'BON'  #'GER'
apps = ['Google_Maps']# ['Instagram','Twitter','Google_Maps'] # 'Instagram'  # 'Twitter' # 'Google_Maps' # 'Facebook'


transfer_modes = ['DL'] # ['DL'] # ['UL'] # ['DL','UL']
type_POIs = ['stadium'] #['stadium','station'] #['stadium','nightclub'] #['stadium']
spatial_units = ['Matmut Stadium Gerland','GER']  #spatial_units = ['Lou_rugby']  # ['Astroballe'] #['Lou_rugby','Ninkasi_Kao'] #['Lou_rugby'] #['Ninkasi_Kao'] 
expanded = '_expanded' # '' # '_expanded' # ''
POI_or_stations = ['POI']# ['POI','station'] # 'station'
modification = {'epochs' : 20, #100
                'lr':1e-4,
                'temporal_h_dim' : 32,
                'spatial_h_dim' : 16,
                'output_h_dim' : 32,
                'TE_embedding_dim': 16,
                'set_spatial_units': ['CHA','GER','BON','SOI'],
                'TE_out_h_dim': 16,
                'TE_concatenation_late':False,
                'TE_concatenation_early':True,
                           }
(trainer,ds,ds_no_shuffle,args) = evaluate_config(model_name,dataset_names,dataset_for_coverage,vision_model_name,transfer_modes= transfer_modes,
                                                   type_POIs = type_POIs,spatial_units = spatial_units,apps = apps,POI_or_stations = POI_or_stations,
                                                   expanded=expanded,modification=modification)


# lr 5e-3
# 32 / 32 / 32 - 3.5e-2   , 380 000
# 32 / 16 / 32 - 3.5e-2   , 380 000

#### Cas `subway_out` stade du Lou Gerland avec un match de Rugby : 

#### Cas `subway_in` Laurent Bonnevay Astroballe avec un match de Basket : 

#### Cas `subway_out` Laurent Bonnevay Astroballe avec un match de Basket : 

## Maintenant on va intégrer les données NetMob et voir si elles permettent de réduire l'erreur de prédiction :

In [1]:
import os 
import pickle 
import pandas as pd 

trial_id = 'subway_in_netmob_POIs_STGCN_VariableSelectionNetwork_MSELoss_2024_12_07_02_55_5679'
#'subway_in_calendar_STGCN_MSELoss_2024_12_12_15_51_46099'
#'subway_in_calendar_STGCN_MSELoss_2024_12_12_14_16_71587'

path_csv = f'save/HyperparameterTuning/{trial_id}.csv'
path_pickle = 'save/HyperparameterTuning/model_args.pkl'
df_hp_tuning = pd.read_csv(path_csv)
model_args = pickle.load(open(path_pickle,'rb'))

metric = '_metric/Loss_model'

best_model = df_hp_tuning.sort_values(metric).iloc[0]
HP_args = [indx.replace('config/', '') for indx in best_model.index if 'config/' in indx]
args = model_args['model'][trial_id]['args']

In [ ]:
print(df_hp_tuning.columns)

df_hp_tuning.sort_values(metric)[['_metric/Loss_model', 'config/lr', 'config/weight_decay',
       'config/dropout', 'config/vision_vision_grn_out_dim']].iloc[:30]

In [ ]:
dataset_names = ["subway_in","netmob_image_per_station"] # ["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'STGCN'
vision_model_name =  'ImageAvgPooling'

(trainer,ds,ds_no_shuffle,args) = evaluate_config(model_name,dataset_names,dataset_for_coverage,vision_model_name)

#### Prediction avec uniquement NetMob, sans subway-in: 

#### Prediction avec uniquement Calendar, sans subway-in: 